# Final Project

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-8551  GCE       4                                       RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby, chain
import pandas as pd
import numpy as np
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
from google.cloud import storage
from contextlib import closing
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict,Counter
from tqdm import tqdm
import operator
import json
from io import StringIO
import math
import matplotlib.pyplot as plt
import hashlib
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  4 19:23 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket-212270458' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh


# Inverted Index

In [ ]:
min_number_to_filter_title=0
min_number_to_filter_body=50
min_number_to_filter_anchor=0

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)

In [ ]:
doc_body_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_preprocessed = parquetFile.select("anchor_text").rdd

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
def map_func(doc_anchor):
  edge_set = []
  for id, anchor in doc_anchor[0]:
    if (id, anchor) not in edge_set:
      edge_set.append((id, anchor))
  return edge_set

doc_anchor_pairs = doc_anchor_preprocessed.flatMap(map_func).distinct().groupByKey().mapValues(list).map(lambda x:(" ".join(x[1]),x[0]))

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  res = {}
  words_count = dict(Counter(tokens))
  for tok in tokens:
    if tok not in all_stopwords:
      res[tok] = (id, words_count[tok])
  return list(res.items())

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key = lambda x: x[0])

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings):
  buckets = postings.map(lambda x: (token2bucket_id(x[0]), x))
  grouped = buckets.groupByKey(NUM_BUCKETS).mapValues(list)
  res = grouped.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))
  return res

def calculate_dl(text, id):
  doc_len = 0
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  for i in tokens:
    if i not in all_stopwords:
      doc_len += 1
  return ((id, doc_len))

def calculate_tt(postings):
  def func(posting):
    sum = 0
    for tup in posting:
      sum += tup[1]
    return sum 
  return postings.mapValues(func)

In [ ]:
word_counts_body = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
postings_body_filtered = postings_body.filter(lambda x: len(x[1])>min_number_to_filter_body)
w2df_body = calculate_df(postings_body_filtered)
w2df_body_dict = w2df_body.collectAsMap()
term_total_body_dict = calculate_tt(postings_body_filtered).collectAsMap()
dl_body_dict = doc_body_pairs.map(lambda x: calculate_dl(x[0], x[1])).collectAsMap()
posting_locs_body_list = partition_postings_and_write(postings_body_filtered).collect()
super_posting_body_locs = defaultdict(list)

word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_title_filtered = postings_title.filter(lambda x: len(x[1])>min_number_to_filter_title)
w2df_title = calculate_df(postings_title_filtered)
w2df_title_dict = w2df_title.collectAsMap()
term_total_title_dict = calculate_tt(postings_title_filtered).collectAsMap()
dl_title_dict = doc_title_pairs.map(lambda x: calculate_dl(x[0], x[1])).collectAsMap()
posting_locs_title_list = partition_postings_and_write(postings_title_filtered).collect()
super_posting_title_locs = defaultdict(list)

word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
postings_anchor_filtered = postings_anchor.filter(lambda x: len(x[1])>min_number_to_filter_anchor)
w2df_anchor = calculate_df(postings_anchor_filtered)
w2df_anchor_dict = w2df_anchor.collectAsMap()
term_total_anchor_dict = calculate_tt(postings_anchor_filtered).collectAsMap()
dl_anchor_dict = doc_anchor_pairs.map(lambda x: calculate_dl(x[0], x[1])).collectAsMap()
posting_locs_anchor_list = partition_postings_and_write(postings_anchor_filtered).collect()
super_posting_anchor_locs = defaultdict(list)

[('political', (23040, 3)), ('philosophy', (23040, 3)), ('social', (23040, 1))]

In [ ]:
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_body_locs[k].extend(v)

for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_title_locs[k].extend(v)

for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_anchor_locs[k].extend(v)

In [ ]:
# Create inverted index instances
index_body = InvertedIndex()
index_body.posting_locs = super_posting_body_locs
index_body.df = w2df_body_dict
index_body.DL = dl_body_dict
index_body.term_total = term_total_body_dict

index_titles = InvertedIndex()
index_titles.posting_locs = super_posting_title_locs
index_titles.df = w2df_title_dict
index_titles.DL = dl_title_dict
index_titles.term_total = term_total_title_dict

index_anchor = InvertedIndex()
index_anchor.posting_locs = super_posting_anchor_locs
index_anchor.df = w2df_anchor_dict
index_anchor.DL = dl_anchor_dict
index_anchor.term_total = term_total_anchor_dict

In [ ]:
index_body.write_index('.','body_index')
index_titles.write_index('.','title_index')
index_anchor.write_index('.', 'anchor_index')

# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

NameError: ignored

In [ ]:
!gsutil ls -lh $index_dst

     181 B  2022-01-04T20:40:48Z  gs://bucket-212270458/postings_gcp/body_index.pkl
TOTAL: 1 objects, 181 bytes (181 B)
